In [1]:
import pandas as pd
import statistics as st

In [2]:
resNuevos = pd.read_csv('resultados_experimentos_modelo_nuevo_30clientes.csv')
resViejos = pd.read_csv('resultados_experimentos_modelo_viejo_30clientes.csv')
resNuevosSinExtra = pd.read_csv('resultados_experimentos_modelo_nuevo_sin_extra_30clientes.csv')

In [7]:
tabla = pd.merge(resNuevos, resViejos, on = 'archivo')
tabla.head()

,archivo,status_x,valor_objetivo_x,tiempo_segundos_x,status_y,valor_objetivo_y,tiempo_segundos_y
0,instancia_0_30_clientes.txt,"integer optimal, tolerance",32956.0,516.492066,"integer optimal, tolerance",46735.0,0.570691
1,instancia_1_30_clientes.txt,integer optimal solution,45083.0,15.775854,"integer optimal, tolerance",45083.0,0.405091
2,instancia_2_30_clientes.txt,"integer optimal, tolerance",44213.0,14.289668,"integer optimal, tolerance",44213.0,0.557126
3,instancia_3_30_clientes.txt,"integer optimal, tolerance",46775.0,38.410404,"integer optimal, tolerance",49476.0,0.494112
4,instancia_4_30_clientes.txt,"integer optimal, tolerance",41920.0,104.886214,"integer optimal, tolerance",43849.0,1.587583


In [8]:
tabla = pd.merge(tabla, resNuevosSinExtra, on='archivo')

In [73]:
tabla.head()

,archivo,status_x,valor_objetivo_x,tiempo_segundos_x,status_y,valor_objetivo_y,tiempo_segundos_y,status,valor_objetivo,tiempo_segundos
0,instancia_0_30_clientes.txt,"integer optimal, tolerance",32956.0,516.492066,"integer optimal, tolerance",46735.0,0.570691,"integer optimal, tolerance",24162.0,9.977224
1,instancia_1_30_clientes.txt,integer optimal solution,45083.0,15.775854,"integer optimal, tolerance",45083.0,0.405091,"integer optimal, tolerance",30881.0,40.948394
2,instancia_2_30_clientes.txt,"integer optimal, tolerance",44213.0,14.289668,"integer optimal, tolerance",44213.0,0.557126,"integer optimal, tolerance",24833.0,619.208348
3,instancia_3_30_clientes.txt,"integer optimal, tolerance",46775.0,38.410404,"integer optimal, tolerance",49476.0,0.494112,integer optimal solution,24448.0,3.337651
4,instancia_4_30_clientes.txt,"integer optimal, tolerance",41920.0,104.886214,"integer optimal, tolerance",43849.0,1.587583,"integer optimal, tolerance",24014.0,84.967986


In [18]:
st.mean(tabla.tiempo_segundos.fillna(6000) - tabla.tiempo_segundos_y) #reemplazo el valor nan por 6000, aunque la realidad es que tardo mas, pero no termino

630.2523614327113

In [82]:
print("Tiempo promedio en resolver una instancia: ")
print(f"Modelo viejo: {tabla.tiempo_segundos_y.mean():.2f}s")
print(f"Modelo nuevo: {tabla.tiempo_segundos.mean():.2f}s (subestima)")
print(f"Modelo nuevo con restricciones extra: {tabla.tiempo_segundos_x.mean():.2f}s")

Tiempo promedio en resolver una instancia: 
Modelo viejo: 38.98s
Modelo nuevo: 485.41s (subestima)
Modelo nuevo con restricciones extra: 43.35s


In [88]:
print(f"En promedio el modelo nuevo tarda {(485.41/38.98 -1)*100:.2f}% mas en resolver una instancia de 30 clientes que el modelo viejo")
print(f"En promedio el modelo nuevo tarda {(485.41/43.35 -1)*100:.2f}% mas en resolver una instancia de 30 clientes que el modelo nuevo con restricciones extra")
print(f"En promedio el modelo nuevo con restricciones extra tarda {(43.35/38.98 -1)*100:.2f}% mas en resolver una instancia de 30 clientes que el modelo viejo")

En promedio el modelo nuevo tarda 1145.28% mas en resolver una instancia de 30 clientes que el modelo viejo
En promedio el modelo nuevo tarda 1019.75% mas en resolver una instancia de 30 clientes que el modelo nuevo con restricciones extra
En promedio el modelo nuevo con restricciones extra tarda 11.21% mas en resolver una instancia de 30 clientes que el modelo viejo


In [85]:
objN = tabla.valor_objetivo.fillna(35070).mean()
objE = tabla.valor_objetivo_x.mean()
objV = tabla.valor_objetivo_y.mean()
print(f"promedio valor objetivo -> modelo viejo: {objV}, modelo restricciones extra: {objE}, modelo nuevo: {objN}")

print(f"El modelo nuevo con restricciones extra reduce el costo promedio en {100 -(objE*100)/objV:.2f} %")
print(f"El modelo nuevo reduce el costo promedio en {100 -(objN*100)/objV:.2f} %")

promedio valor objetivo -> modelo viejo: 45601.63333333333, modelo restricciones extra: 40200.833333333336, modelo nuevo: 29413.233333333334
El modelo nuevo con restricciones extra reduce el costo promedio en 11.84 %
El modelo nuevo reduce el costo promedio en 35.50 %
